In [ ]:
import numpy as np
import xarray as xr
import glidertools as gt

In [ ]:
# Open data

# open Surface fields
fname_SSH = "./data/SSH_sogos.nc"
fname_FSLE = "./data/FSLE_sogos.nc"

ds_ssh = xr.open_dataset(fname_SSH)
ds_fsle = xr.open_dataset(fname_FSLE)

# Create variables for vectors
ds_ssh['mag'] = np.sqrt(ds_ssh.ugos**2 + ds_ssh.vgos**2)
ds_ssh['angle'] = (np.pi/2.) - np.arctan2(ds_ssh.ugos/ds_ssh['mag'], 
                                          ds_ssh.vgos/ds_ssh['mag'])

In [ ]:
# open glider files
glid_folder = './data/'
ds_659 = xr.load_dataset(glid_folder + 'sg659/CTD_659.nc')
ds_660 = xr.load_dataset(glid_folder + 'sg660/CTD_660.nc')

# easier to work with a days variable that is a float rather than datenum
days = ds_659.time - np.datetime64('2019-01-01')
ds_659['days'] = (days / np.timedelta64(1, 'D'))

days = ds_660.time - np.datetime64('2019-01-01')
ds_660['days'] = (days / np.timedelta64(1, 'D'))

# Group and average by dives so that plotting of positions is fast
ds_659_av = ds_659.groupby('dives').mean()
ds_660_av = ds_660.groupby('dives').mean()

ds_659_av = ds_659_av.drop('dives')
ds_660_av = ds_660_av.drop('dives')

In [ ]:
ds_659

In [ ]:
ds_659['potdens'] = gt.physics.potential_density(ds_659.salinity, 
                                                 ds_659.temperature, 
                                                 ds_659.pressure, 
                                                 ds_659.latitude, 
                                                 ds_659.longitude)

ds_660['potdens'] = gt.physics.potential_density(ds_660.salinity, 
                                                 ds_660.temperature, 
                                                 ds_660.pressure,
                                                 ds_660.latitude, 
                                                 ds_660.longitude)

# we can add mixed layer depth, N2 etc in the future versions

In [ ]:
ds_659['spice'] = gt.physics.spice0(ds_659.salinity, 
                         ds_659.temperature, 
                         ds_659.pressure, 
                         ds_659.latitude, 
                         ds_659.longitude)

ds_660['spice'] = gt.physics.spice0(ds_660.salinity, 
                         ds_660.temperature, 
                         ds_660.pressure, 
                         ds_660.latitude, 
                         ds_660.longitude)

In [ ]:
# Interpolate and grid glider data on pressure-time axis
# There are many ways this can be done. We choose a simple linear interpolation in time and pressure
# We could alternatively interpolate in density-time, pressure-distance, dive-pressure etc.
# This a place where a lot of more work into GP and 
# learning the most optimal interpolation (in sense of MLE) might work. 

# Note this is different from what glidertools does, which does a simple binning. 

from scipy.interpolate import griddata
# interpolate on pressure-time 
def interp_pres_time(ds_glid, var): 
    pres_ug = ds_glid.pressure
    time_ug = ds_glid.days
    
    # convert to points values
    points = np.stack([time_ug.values, pres_ug.values],
                       axis=1)
    values = ds_glid[var].values
    
    # remove nans
    non_nan = np.logical_and(np.logical_and(~np.isnan(points[:,0]), 
                                      ~np.isnan(points[:,1])),
                                      ~np.isnan(values))
    
    points =points[non_nan,:]
    values =values[non_nan]
    
    # define grid
    pres_grid = np.linspace(0,1000,1001)
    time_grid = np.arange(119, 207, 0.025)
    grid_p, grid_t = np.meshgrid(pres_grid, time_grid)
    
    temp_grided = griddata(points, values, 
                         (grid_t, grid_p), 
                         method='linear', rescale=True)
    
    return xr.DataArray(temp_grided.T, 
                               dims=["pressure", "time"],
                          coords={"pressure":pres_grid,
                                    "time":time_grid}).rename(var)

# apply to all useful glider variables 
# can later add in variables measured by other instruments too 
def convert_glider(ds_glid, vars_convert= ['temperature','salinity','potdens','spice']):
    
    #vars_convert = ['temperature','salinity','potdens']
    
    ds_grid = xr.Dataset()
    
    for v in vars_convert:
            ds_grid[v] = interp_pres_time(ds_glid, v)
    
    return ds_grid

In [ ]:
# convert from point data to gridded data, can take some time
ds_659_grid = convert_glider(ds_659)
ds_660_grid = convert_glider(ds_660)

In [ ]:
data_settings = {"zlib": True, "dtype":'float32', "complevel": 9}
encoding_dict = dict(temperature=data_settings,
                     salinity=data_settings,
                     potdens=data_settings,
                     spice=data_settings)

ds_659_grid.to_netcdf('659_grid.nc', encoding=encoding_dict)

ds_660_grid.to_netcdf('660_grid.nc', encoding=encoding_dict)